## PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook
for the following questions.


In [15]:
# Establish connection with SQLite Database
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///data/country_club_db.sqlite')

In [22]:
# snapshot of Members table
df_members = pd.read_sql_query("SELECT * FROM Members", engine)
df_members.head(2)

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05


In [24]:
# snapshot of Facilities table
df_facilities = pd.read_sql_query("SELECT * FROM Facilities", engine)
df_facilities.head(2)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200


In [25]:
# snapshot of Bookings table
df_Bookings = pd.read_sql_query("SELECT * FROM Bookings", engine)
df_Bookings.head(2)

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2



### QUESTIONS:
**Q10**: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!


In [75]:
# Construct Query
query_10 = '''
SELECT name 
       , tot_revenue 
FROM   (SELECT f.name 
               , Sum(CASE 
                       WHEN b.memid = 0 THEN f.guestcost * b.slots 
                       ELSE f.membercost * b.slots 
                     end) AS tot_revenue 
        FROM   bookings AS b 
               INNER JOIN facilities AS f 
                       ON b.facid = f.facid 
        GROUP  BY f.name) 
WHERE  tot_revenue > 1000 
ORDER  BY tot_revenue DESC; 
'''

In [76]:
# Execute Query
q10 = pd.read_sql_query(query_10, engine)

In [77]:
# Display Result
q10

,name,tot_revenue
0,Massage Room 1,50351.6
1,Massage Room 2,14454.6
2,Tennis Court 2,14310.0
3,Tennis Court 1,13860.0
4,Squash Court,13468.0
5,Badminton Court,1906.5


**Q11**: Produce a report of members and who recommended them in alphabetic surname,firstname order


In [235]:
# Construct Query
query_11 = '''
SELECT m1.surname 
       , m1.firstname 
       , m2.surname   AS referral_surname 
       , m2.firstname AS referral_firstname 
FROM   members AS m1 
       INNER JOIN members AS m2 
               ON m1.recommendedby = m2.memid 
ORDER  BY m1.surname 
          , m1.firstname; 
'''

In [236]:
# Execute Query
q11 = pd.read_sql_query(query_11, engine)

In [237]:
# Display Result
q11

,surname,firstname,referral_surname,referral_firstname
0,Bader,Florence,Stibbons,Ponder
1,Baker,Anne,Stibbons,Ponder
2,Baker,Timothy,Farrell,Jemima
3,Boothe,Tim,Rownam,Tim
4,Butters,Gerald,Smith,Darren
5,Coplin,Joan,Baker,Timothy
6,Crumpet,Erica,Smith,Tracy
7,Dare,Nancy,Joplette,Janice
8,Genting,Matthew,Butters,Gerald
9,Hunt,John,Purview,Millicent


**Q12**: Find the facilities with their usage by member, but not guests 

In [238]:
# Construct Query
query_12 = '''
SELECT f.name           AS facility 
       , Count(b.facid) AS times_used_by_members 
FROM   bookings AS b 
       INNER JOIN facilities AS f 
               ON b.facid = f.facid 
WHERE  b.memid != 0 
GROUP  BY b.facid 
ORDER  BY b.facid DESC; 
'''

In [239]:
# Execute Query
q12 = pd.read_sql_query(query_12, engine)

In [240]:
# Display Result
q12

,facility,times_used_by_members
0,Pool Table,783
1,Snooker Table,421
2,Squash Court,195
3,Massage Room 2,27
4,Massage Room 1,421
5,Table Tennis,385
6,Badminton Court,344
7,Tennis Court 2,276
8,Tennis Court 1,308


Counting the hours instead of the number of reservations is more representative of the utilization of the facility.

In [241]:
# Construct Query
# This time let's count the number of hours the facility has been used by members
query_12b = '''
SELECT f.name             AS facility 
       , Sum(b.slots) / 2 AS hours_used_by_member 
FROM   bookings AS b 
       INNER JOIN facilities AS f 
               ON b.facid = f.facid 
WHERE  b.memid != 0 
GROUP  BY b.facid 
ORDER  BY hours_used_by_member DESC; 
'''

In [242]:
# Execute Query
q12b = pd.read_sql_query(query_12b, engine)

In [243]:
# Display Result
q12b

,facility,hours_used_by_member
0,Badminton Court,543
1,Tennis Court 1,478
2,Massage Room 1,442
3,Tennis Court 2,441
4,Snooker Table,430
5,Pool Table,428
6,Table Tennis,397
7,Squash Court,209
8,Massage Room 2,27


**Q13**: Find the facilities usage by month, but not guests

In [244]:
# Construct Query
query_13 = '''
SELECT f.name                        AS facility 
       , Count(b.slots) / 2          AS member_usage_hours 
       , Strftime('%m', b.starttime) AS reservation_month 
FROM   bookings AS b 
       INNER JOIN facilities AS f 
               ON b.facid = f.facid 
WHERE  b.memid != 0 
GROUP  BY f.name 
          , reservation_month 
ORDER  BY reservation_month DESC 
          , member_usage_hours DESC 
          , facility; 
'''

In [245]:
# Execute Query
q13 = pd.read_sql_query(query_13, engine)

In [246]:
# Display Result
q13

,facility,member_usage_hours,reservation_month
0,Pool Table,204,09
1,Snooker Table,99,09
2,Table Tennis,97,09
3,Massage Room 1,95,09
4,Badminton Court,80,09
5,Tennis Court 1,66,09
6,Tennis Court 2,63,09
7,Squash Court,43,09
8,Massage Room 2,7,09
9,Pool Table,136,08
